# Optimization in Python

[Source](https://www.youtube.com/watch?v=cXHvC_FGx24)

#### Objective

**Minimize**

$x_1.x_4.(x_1 + x_2 + x_3) + x_3$

**Such that**

Constraints:

* #1:
    * $x_1.x_2.x_3.x_4 ≥ 25$

* #2:
    * $x_1^2 + x_2^2 + x_3^2 + x_4^2 = 40$

* #3:
    * $1 ≤ x_1,x_2,x_3,x_4 ≤ 5$

Example

$x_0 = (1, 5, 5, 1)$



In [2]:
import numpy as np
from scipy.optimize import minimize

In [3]:
def objective(x):
    [x1, x2, x3, x4] = x
    return x1 * x4 * (x1 + x2 + x3) + x3

def constraint1(x):
    [x1, x2, x3, x4] = x
    return x1 * x2 * x3 * x4 - 25.0

def constraint2(x):
    sum_sq = 40
    for i in range(4):
        sum_sq = sum_sq - x[i] ** 2

    return sum_sq

In [4]:
# initial guess
x0 = [1, 5, 5, 1]

objective(x0)

16

In [5]:
bounds = ((1.0, 5.0),) * len(x0)

bounds

((1.0, 5.0), (1.0, 5.0), (1.0, 5.0), (1.0, 5.0))

In [6]:
constraints = [{
    'type': 'ineq',
    'fun': constraint1
},{
    'type': 'eq',
    'fun': constraint2
}]

In [7]:
solution = minimize(objective, x0, method='SLSQP',bounds=bounds, constraints=constraints)

solution

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 17.01401724563517
       x: [ 1.000e+00  4.743e+00  3.821e+00  1.379e+00]
     nit: 5
     jac: [ 1.457e+01  1.379e+00  2.379e+00  9.564e+00]
    nfev: 25
    njev: 5

In [8]:
print(solution["fun"])

solution["x"]

17.01401724563517


array([1.        , 4.7429961 , 3.82115462, 1.37940765])

In [9]:
from functools import reduce

constraint1_passed = reduce(lambda x, y: x * y, solution["x"])

constraint1_passed


24.99999994523914

In [10]:


constraint2_passed = 0

for x in solution["x"]:
    constraint2_passed += x ** 2

constraint2_passed

40.00000008235324

# Intro to Scipy Optimization: Minimize Method

[Source](https://www.youtube.com/watch?v=G0yP_TM-oag)

#### Problem 1

**Objective**

We want to find the lowest point on the curve:

$f(x) = x^2 - 12x + 20$

That would be to get the roots of $f(x)$ i.e $x$ values that return $f(x) = 0$

In [11]:
from scipy.optimize import minimize

In [12]:
# function to minimize
def f(x):
    y = x ** 2 - 12 * x + 20
    return y

# starting guess
x_start = 2.0

# optimizing to get the corresponding x value for the lowest value on the curve
result = minimize(f, x_start, options={"disp": True})

result

Optimization terminated successfully.
         Current function value: -16.000000
         Iterations: 3
         Function evaluations: 8
         Gradient evaluations: 4


  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -15.999999999999936
        x: [ 6.000e+00]
      nit: 3
      jac: [ 4.768e-07]
 hess_inv: [[ 5.000e-01]]
     nfev: 8
     njev: 4

In [13]:
# f(x) value that is the lowest on the curve is stored in "fun" key in the result
result["fun"]

-15.999999999999936

In [14]:
# x is the value(s) that when plugged into our objective function gives us the resulting "fun" i.e f(x)
result["x"]

array([6.00000025])

#### Let's try to get the roots of $f(x)$

In [15]:
bounds = ((0, 0),)

minimize(f, x_start, method="SLSQP", constraints=[
    {
        "type":"eq",
        'fun': lambda x: 0
    }
], options={"disp": True})

Singular matrix C in LSQ subproblem    (Exit mode 6)
            Current function value: 0.0
            Iterations: 1
            Function evaluations: 2
            Gradient evaluations: 1


 message: Singular matrix C in LSQ subproblem
 success: False
  status: 6
     fun: 0.0
       x: [ 2.000e+00]
     nit: 1
     jac: [-8.000e+00]
    nfev: 2
    njev: 1

#### Problem 2

**Maximizing the Area of a Garden**

*A rectangular garden is to be constructed using a rock wall as one side of the garden and wire fencing for the other three sides. Given $100ft$ of wire fencing, determine the dimensions that would create a garden of maximum area. What is the maximum area?*

In [16]:
# function to minimize (maximize area)
def f(dimension):
    [x, y] = dimension
    area = x * y
    return -area

# starting guess
dimension_guess = [50, 50]

constraints = [
    {
        "type": "eq",
        "fun": lambda dimension: 100 - (2 * dimension[0] + dimension[1])
    }
]

result = minimize(f, dimension_guess, method="SLSQP",bounds=((1, 100),(1, 100)), constraints=constraints, options={"disp":True})

result

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1250.0000000000002
            Iterations: 4
            Function evaluations: 12
            Gradient evaluations: 4


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -1250.0000000000002
       x: [ 2.500e+01  5.000e+01]
     nit: 4
     jac: [-5.000e+01 -2.500e+01]
    nfev: 12
    njev: 4

In [17]:
result["x"]

array([25., 50.])

In [18]:
-result["fun"]

1250.0000000000002

#### adding another constraint

If we add another constraint i.e *the side (y) adjacent to the rock wall is limited to 30, what would be the maximum area*

In [19]:
constraints = [
    {
        "type": "eq",
        "fun": lambda dimension: 100 - (2 * dimension[0] + dimension[1])
    },
    {
        "type": "eq",
        "fun": lambda dimension: 30 - dimension[1]
    }
]

result = minimize(f, dimension_guess, method="SLSQP", bounds=(
    (1, 100), (1, 100)), constraints=constraints, options={"disp": True})

result

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1050.0
            Iterations: 2
            Function evaluations: 6
            Gradient evaluations: 2


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -1050.0
       x: [ 3.500e+01  3.000e+01]
     nit: 2
     jac: [-3.000e+01 -3.500e+01]
    nfev: 6
    njev: 2

In [20]:
result.x

array([35., 30.])

In [21]:
result.fun

-1050.0

### Problem 3

$$f(x) = x^4 - 8x^2 + 5$$

**Objective**

Find the minima and maxima

**Boundaries**

* $x = [-1, 3]$

Let's refer to some rules for differentiation

$f^{'}(x) = 0$ at a turning point

**Constraints**

* $f^{'}(x) = 0$

#### Minima

In [22]:
def f(x):
    return x ** 4 - (8 * x ** 2) + 5

init_guess = 20

bounds = ((-1, 3),)

constraints = [
    {
        'type': 'eq',
        'fun': lambda x: 4 * x**3 - 16 * x
    }
]

result = minimize(f, init_guess, bounds=bounds, constraints=constraints, options={"disp": True})

result

Optimization terminated successfully    (Exit mode 0)
            Current function value: -11.0
            Iterations: 5
            Function evaluations: 11
            Gradient evaluations: 5


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -11.0
       x: [ 2.000e+00]
     nit: 5
     jac: [ 2.031e-04]
    nfev: 11
    njev: 5

#### Maxima

In [23]:
def f(x):
    return x ** 4 - (8 * x ** 2) + 5


init_guess = 20

bounds = ((-1, 1),)

constraints = [
    {
        'type': 'eq',
        'fun': lambda x: 4 * x**3 - 16 * x
    }
]

result = minimize(f, init_guess, bounds=bounds,
                  constraints=constraints, options={"disp": True})

result

Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.0
            Iterations: 2
            Function evaluations: 5
            Gradient evaluations: 2


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 5.0
       x: [ 0.000e+00]
     nit: 2
     jac: [-1.192e-07]
    nfev: 5
    njev: 2

#### Problem 4

$$f(x) = 2x^4 - 3x^3 + 5x^2 - 2x + 1$$

**Objective**

Find the minima and maxima of the above function

**Constraints**

* x-values when plugged into $f^{'}(x) = 0$
    * $f^{'}(x) = 8x^3 - 9x^2 + 10x - 2$

In [25]:
def f(x):
    return 2 * (x ** 4) - 3 * (x ** 3) + 5 * (x ** 2) - 2*x + 1

constraints = [
    {
        'type': 'eq',
        'fun': lambda x: 8 * (x ** 3) - 9 * (x ** 2) + 10 * x - 2
    }
]

init_guess = 0

result = minimize(f, init_guess, constraints=constraints)

result

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.7731592023847624
       x: [ 2.411e-01]
     nit: 4
     jac: [-2.488e-06]
    nfev: 9
    njev: 4

In [28]:
print(f"x = {result.x}\n\nf(x)={result.fun}")

x = [0.24110613]

f(x)=0.7731592023847624


In [29]:
def f(x):
    return -(2 * (x ** 4) - 3 * (x ** 3) + 5 * (x ** 2) - 2*x + 1)


constraints = [
    {
        'type': 'eq',
        'fun': lambda x: 8 * (x ** 3) - 9 * (x ** 2) + 10 * x - 2
    }
]

init_guess = 0

result = minimize(f, init_guess, constraints=constraints)

result

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.7731592023847624
       x: [ 2.411e-01]
     nit: 4
     jac: [ 2.488e-06]
    nfev: 9
    njev: 4